# Testinnsending av upersonlig skattemelding for midlertidig lagring

Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan opprette en midlertidig lagret upersonlig skattemelding.

In [ ]:
try:
    from skatteetaten_api import get_access_token, decode_dokument
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
except ImportError as e:
    print("Mangler en eller avhengighet, installer dem via pip, se requierments.txt fil for detaljer")
    raise ImportError(e)


## Variabler som må endres i test
Før testscriptet kjøres trengs det bare å definere URLer som skal brukes.

In [ ]:
skatteetaten_url = "https://idporten-api-sbstest.sits.no"

fnr="12877597928" #oppdater med test fødselsnummerene du har fått tildelt
orgnr_as = "312787016"

# oppdater med inntektsaar innsendingen gjelder for
inntektsaar = 2023

## Generer ID-porten token
Tokenet er gyldig i 300 sekunder, rekjørt denne biten om du ikke har nådd Altinn3 biten innen 300 sekunder

In [ ]:
idporten_header = get_access_token()

# Hent utkast og gjeldende
Her legger vi inn fødselsnummeret vi logget oss inn med. Dersom du velger et annet fødselsnummer så må den du logget på med ha tilgang til skattemeldingen du ønsker å hente

Parten nedenfor er brukt for internt test, pass på bruk deres egne testparter når dere tester


In [ ]:
s = requests.Session()
s.headers = dict(idporten_header)

### Utkast

In [ ]:
url_gjeldende = f'{skatteetaten_url}/api/skattemelding/v2/{inntektsaar}/{orgnr_as}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende

In [ ]:
print(r_gjeldende.text)

## Svar fra hent gjeldende 

### Gjeldende dokument referanse: 
I responsen på alle api kallene, være seg utkast/fastsatt eller gjeldene, så følger det med en dokumentreferanse. 
For å kalle valider tjenesten, er en avhengig av å bruke korrekt referanse til gjeldende skattemelding. 

Cellen nedenfor henter ut gjeldende dokumentrefranse printer ut responsen fra hent gjeldende kallet 

In [ ]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
sme_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']
decoded_sme_xml = decode_dokument(skattemelding_base64)
sme_utkast = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
sme_gjeldende_dict = xmltodict.parse(sme_utkast)
partsnummer = sme_gjeldende_dict["skattemelding"]["partsnummer"]

print(f"Responsen fra hent gjeldende ser slik ut, gjeldende dokumentrerefanse er {dokref}")
print(sjekk_svar.request.method ,sjekk_svar.request.url)
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())


In [ ]:
#Legg merge til dokumenter.dokument.type = skattemeldingUpersonlig

with open(f"../../../src/resources/eksempler/{inntektsaar}/Naeringspesifikasjon-AS-v3.xml", 'r') as f:
    naering_as_xml = f.read()
    
innsendingstype = "komplett"
naeringsspesifikasjoner_as_b64 = base64.b64encode(naering_as_xml.encode("utf-8"))
naeringsspesifikasjoner_as_b64 = str(naeringsspesifikasjoner_as_b64.decode("utf-8"))
naeringsspesifikasjoner_base64=naeringsspesifikasjoner_as_b64


with open(f"../../../src/resources/eksempler/{inntektsaar}/upersonligSkattemeldingV2.xml", 'r') as f:
    sme_verdi_bak_aksjer = f.read().format(partsnummer=partsnummer)
    sme_b64 = base64.b64encode(sme_verdi_bak_aksjer.encode("utf-8"))
    sme_b64 = str(sme_b64.decode("utf-8"))


valider_konvlutt_v2 = """
<?xml version="1.0" encoding="utf-8" ?>
<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingUpersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjon</type>
            <encoding>utf-8</encoding>
            <content>{naeringsspeifikasjon_base64}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingUpersonlig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>{inntektsaar}</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>{innsendingstype}</innsendingstype>
        <opprettetAv>TurboSkatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>
""".replace("\n","")


naering_as = valider_konvlutt_v2.format(sme_base64=sme_b64,
                                         naeringsspeifikasjon_base64=naeringsspesifikasjoner_base64,
                                         dok_ref=dokref,
                                         inntektsaar=inntektsaar,
                                         innsendingstype=innsendingstype)

# Valider utkast sme med næringsopplysninger

In [ ]:
def valider_sme(payload):
    url_valider = f'{skatteetaten_url}/api/skattemelding/v2/valider/{inntektsaar}/{orgnr_as}'
    header = dict(idporten_header)
    header["Content-Type"] = "application/xml"
    return s.post(url_valider, headers=header, data=payload)

valider_respons = valider_sme(naering_as)
resultatAvValidering = xmltodict.parse(valider_respons.text)["skattemeldingOgNaeringsspesifikasjonResponse"]["resultatAvValidering"]

if valider_respons:
    print(resultatAvValidering + "\n")
    print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())
else:
    print(valider_respons.status_code, valider_respons.headers, valider_respons.text)

## Midlertidig lagret skattemelding
I dette steget sender vi inn en skattemelding-xml som er opprettet av et sluttbrukersystem. Innsendingen vil også bli validert. For å kunne opprette en midlertidig lagret skattemelding for selskapet kreves det at du har en gyldig rolle i Altinn.

In [ ]:
header = dict(idporten_header)
header["Content-Type"] = "application/xml"
midlertidig_lagring_path = "/api/skattemelding/v2/til-midlertidig-lagret-skattemelding-for-visning-upersonlig"
r = s.post(f"{skatteetaten_url}{midlertidig_lagring_path}/{orgnr_as}", data=naering_as, headers=header)
r

### Print link til midlertidig lagret skattemelding
I dette steget kan du trykke på lenken som åpner visning av utkastet. Alle som har en gyldig rolle i Altinn mot selskapet vil kunne åpne denne lenken.

In [ ]:
print(r.json()['url'])